In [1]:
A0 =0x67452301
B0 = 0xefcdab89
C0 = 0x98badcfe
D0 = 0x10325476
K = [
    0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
    0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
    0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
    0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821,
    0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa,
    0xd62f105d, 0x02441453, 0xd8a1e681, 0xe7d3fbc8,
    0x21e1cde6, 0xc33707d6, 0xf4d50d87, 0x455a14ed,
    0xa9e3e905, 0xfcefa3f8, 0x676f02d9, 0x8d2a4c8a,
    0xfffa3942, 0x8771f681, 0x6d9d6122, 0xfde5380c,
    0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70,
    0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x04881d05,
    0xd9d4d039, 0xe6db99e5, 0x1fa27cf8, 0xc4ac5665,
    0xf4292244, 0x432aff97, 0xab9423a7, 0xfc93a039,
    0x655b59c3, 0x8f0ccc92, 0xffeff47d, 0x85845dd1,
    0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1,
    0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391
]
s = [
    7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21
]

In [ ]:
# d1=b"abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00"
# def bytes_to_binary_string(byte_data):
#     binary_str = ''.join(f'{byte:08b} ' for byte in byte_data)  # Convert each byte to 8-bit binary
#     return binary_str.strip()
# # padded_output = b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00'
# binary_representation = bytes_to_binary_string(d1)
# print(binary_representation.replace(" ",""))

In [39]:
def number_to_binary_little_endian(n):
    """Convert an integer to a 64-bit binary string in little-endian format."""
    return ''.join(format((n >> (8 * i)) & 0xFF, '08b') for i in range(8))

def preprocessing_input(input):
    """Prepares input for MD5 by adding padding and length encoding (little-endian)."""
    input_bytes = input.encode('utf-8')
    n = len(input_bytes)
    # Convert message to binary
    message_bits = ''.join(format(byte, '08b') for byte in input_bytes)
    # Append '1' bit
    message_bits += '1'
    # Compute k (number of zero bits required)
    k = (448 - ((n * 8 + 1) % 512)) % 512
    message_bits += '0' * k  # Append k zero bits
    # Append length in 64-bit LITTLE-ENDIAN format
    length_bits = number_to_binary_little_endian(n * 8)
    message_bits += length_bits
    return message_bits

print(preprocessing_input('abc'))

True


In [4]:
def F(x,y,z):
  return (x & y) | ((~x) & z)
def G(x,y,z):
   return (x & z) | (y & (~z))
def H(x,y,z):
   return x^y^z
def I(x,y,z):
    return y ^ (x | (~z))
def left_rotate(n, b):
    """Left rotate a 32-bit integer n by b bits."""
    return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF    

In [5]:
def generate_words(binary_input):
    words = []
    for i in range(16):  # 16 words of 32 bits each
        start = i * 32
        end = start + 32
        word = binary_input[start:end]  # Extract 32-bit chunk
        # Convert binary string to 4 bytes (each byte = 8 bits)
        byte1 = word[0:8]
        byte2 = word[8:16]
        byte3 = word[16:24]
        byte4 = word[24:32]
        # Reverse byte order (little-endian conversion)
        little_endian_binary = byte4 + byte3 + byte2 + byte1  # Swap positions
        # Convert back to integer
        int_word = int(little_endian_binary, 2)
        words.append(int_word)
        # words.append(little_endian_binary)
    return words
M=generate_words(preprocessing_input('abc'))    
# for i in range(len(M)):
    # M[i]=M[i][::-1]
print(M)    
# M=[2153996897, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0]

[2153996897, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0]


In [ ]:
# def int_list_to_binary_strings(int_list):
#     binary_strings = []
#     for num in int_list:
#         # Convert to 32-bit binary string, padded with leading zeros
#         binary_str = format(num, '032b')
#         binary_strings.append(binary_str)
#     return binary_strings

# A33 = [29771398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0]
# binary_list = int_list_to_binary_strings(A33)

# # Print the result
# print(binary_list)
# print(binary_list==M)

In [7]:
A = A0
B = B0
C = C0
D = D0

for i in range(64):
    if 0 <= i <= 15:
        f = F(B, C, D)
        g = i
    elif 16 <= i <= 31:
        f = G(B, C, D)
        g = (5 * i + 1) % 16
    elif 32 <= i <= 47:
        f = H(B, C, D)
        g = (3 * i + 5) % 16
    elif 48 <= i <= 63:
        f = I(B, C, D)
        g = (7 * i) % 16

    # Save original A value
    temp_A = A  
    
    # Compute new value
    temp = (A + f + K[i] + M[g]) & 0xFFFFFFFF  # Mod 2^32
    temp = left_rotate(temp, s[i])
    temp = (B + temp) & 0xFFFFFFFF  # Mod 2^32

    # Shift registers cyclically
    A = D
    D = C
    C = B
    B = temp  # Updated B value

# Update initial hash values
A0 = (A0 + A) & 0xFFFFFFFF
B0 = (B0 + B) & 0xFFFFFFFF
C0 = (C0 + C) & 0xFFFFFFFF
D0 = (D0 + D) & 0xFFFFFFFF


In [8]:
print(hex(A0)+hex(B0)+hex(C0)+hex(D0)=="0x985001900xb04fd23c0x7d3f96d60x727fe128")
print(A0)

True
2555380112


In [ ]:
# Initial state variables
A0 = 0x67452301
B0 = 0xefcdab89
C0 = 0x98badcfe
D0 = 0x10325476

# Constants
K = [
    0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
    0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
    0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
    0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821,
    0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa,
    0xd62f105d, 0x02441453, 0xd8a1e681, 0xe7d3fbc8,
    0x21e1cde6, 0xc33707d6, 0xf4d50d87, 0x455a14ed,
    0xa9e3e905, 0xfcefa3f8, 0x676f02d9, 0x8d2a4c8a,
    0xfffa3942, 0x8771f681, 0x6d9d6122, 0xfde5380c,
    0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70,
    0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x04881d05,
    0xd9d4d039, 0xe6db99e5, 0x1fa27cf8, 0xc4ac5665,
    0xf4292244, 0x432aff97, 0xab9423a7, 0xfc93a039,
    0x655b59c3, 0x8f0ccc92, 0xffeff47d, 0x85845dd1,
    0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1,
    0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391
]

# Shift amounts
s = [
    7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21
]
# Preprocessing function
def number_to_binary_little_endian(n):
    """Convert an integer to a 64-bit binary string in little-endian format."""
    return ''.join(format((n >> (8 * i)) & 0xFF, '08b') for i in range(8))

def preprocessing_input(input):
    """Prepares input for MD5 by adding padding and length encoding (little-endian)."""
    input_bytes = input.encode('utf-8')
    n = len(input_bytes)

    # Convert message to binary
    message_bits = ''.join(format(byte, '08b') for byte in input_bytes)

    # Append '1' bit
    message_bits += '1'

    # Compute k (number of zero bits required)
    k = (448 - ((n * 8 + 1) % 512)) % 512
    message_bits += '0' * k  # Append k zero bits

    # Append length in 64-bit LITTLE-ENDIAN format
    length_bits = number_to_binary_little_endian(n * 8)
    message_bits += length_bits

    return message_bits

# print(preprocessing_input('abc'))

# Logical functions
def F(x, y, z):
    return (x & y) | (~x & z)

def G(x, y, z):
    return (x & z) | (y & ~z)

def H(x, y, z):
    return x ^ y ^ z

def I(x, y, z):
    return y ^ (x | ~z)

# Left rotate function
def left_rotate(n, b):
    return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

# Word generation function
def generate_words(binary_input):
    words = []
    for i in range(16):  # 16 words of 32 bits each
        start = i * 32
        end = start + 32
        word = binary_input[start:end]  # Extract 32-bit chunk
        # Convert binary string to 4 bytes (each byte = 8 bits)
        byte1 = word[0:8]
        byte2 = word[8:16]
        byte3 = word[16:24]
        byte4 = word[24:32]
        # Reverse byte order (little-endian conversion)
        little_endian_binary = byte4 + byte3 + byte2 + byte1  # Swap positions
        # Convert back to integer
        int_word = int(little_endian_binary, 2)
        words.append(int_word)
    return words
print(generate_words(preprocessing_input('abc')))
# Main MD5 computation
def md5(input):
    # Preprocess input
    padded_message = preprocessing_input(input)
    M = generate_words(padded_message)
    # Initialize state variables
    A = A0
    B = B0
    C = C0
    D = D0

    # Main loop
    for i in range(64):
        if 0 <= i < 16:
            f = F(B, C, D)
            g = i
        elif 16 <= i < 32:
            f = G(B, C, D)
            g = (5 * i + 1) % 16
        elif 32 <= i < 48:
            f = H(B, C, D)
            g = (3 * i + 5) % 16
        elif 48 <= i < 64:
            f = I(B, C, D)
            g = (7 * i) % 16

        # Compute intermediate value
        temp = (A + f + K[i] + M[g]) & 0xFFFFFFFF  # Mod 2^32
        temp = left_rotate(temp, s[i])
        temp = (B + temp) & 0xFFFFFFFF  # Mod 2^32

        # Update state variables
        A, B, C, D = D, temp, B, C  # Correct order: A = D, D = C, C = B, B = temp

    # Final addition modulo 2^32
    A_final = (A0 + A) & 0xFFFFFFFF
    B_final = (B0 + B) & 0xFFFFFFFF
    C_final = (C0 + C) & 0xFFFFFFFF
    D_final = (D0 + D) & 0xFFFFFFFF
    print(hex(A_final)+hex(B_final)+hex(C_final)+hex(D_final)) 
    # Convert state variables to little-endian bytes and concatenate
    hash_bytes = (
        A_final.to_bytes(4, 'little') +
        B_final.to_bytes(4, 'little') +
        C_final.to_bytes(4, 'little') +
        D_final.to_bytes(4, 'little')
    )

    # Return as hex string
    return hash_bytes.hex()

print(md5('abc')) 
print("900150983cd24fb0d6963f7d28e17f72"=="900150983cd24fb0d6963f7d28e17f72")

[2153996897, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0]
0x985001900xb04fd23c0x7d3f96d60x727fe128
900150983cd24fb0d6963f7d28e17f72
True


In [ ]:
import struct

# Initial state variables
A0 = 0x67452301
B0 = 0xefcdab89
C0 = 0x98badcfe
D0 = 0x10325476

# Constants
K = [
    0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
    0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
    0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
    0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821,
    0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa,
    0xd62f105d, 0x02441453, 0xd8a1e681, 0xe7d3fbc8,
    0x21e1cde6, 0xc33707d6, 0xf4d50d87, 0x455a14ed,
    0xa9e3e905, 0xfcefa3f8, 0x676f02d9, 0x8d2a4c8a,
    0xfffa3942, 0x8771f681, 0x6d9d6122, 0xfde5380c,
    0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70,
    0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x04881d05,
    0xd9d4d039, 0xe6db99e5, 0x1fa27cf8, 0xc4ac5665,
    0xf4292244, 0x432aff97, 0xab9423a7, 0xfc93a039,
    0x655b59c3, 0x8f0ccc92, 0xffeff47d, 0x85845dd1,
    0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1,
    0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391
]

# Shift amounts
s = [
    7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21
]

# Logical functions
def F(x, y, z):
    return (x & y) | (~x & z)

def G(x, y, z):
    return (x & z) | (y & ~z)

def H(x, y, z):
    return x ^ y ^ z

def I(x, y, z):
    return y ^ (x | ~z)

# Left rotate function
def left_rotate(n, b):
    return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

# Preprocessing function
def preprocessing_input(input):
    input_bytes = input.encode('utf-8')  # Convert to bytes
    n = len(input_bytes)  # Length in bytes
    # Append the bit '1' to the message
    input_bytes += b'\x80'
    # Append 0 ≤ k < 512 bits so that the resulting message length is congruent to 448 mod 512
    k = (56 - (n + 1) % 64) % 64  # 56 bytes = 448 bits
    input_bytes += b'\x00' * k
    # Append the original length in bits (little-endian)
    input_bytes += struct.pack('<Q', n * 8)  # '<Q' means little-endian unsigned long long
    return input_bytes
# a11=preprocessing_input('abc');
# print(a11)
# MD5 compression function
def md5_compress(A, B, C, D, block):
    # Unpack the block into 16 32-bit little-endian words
    print(block,' block .. ')
    words = struct.unpack('<16I', block)  # '<16I' means 16 little-endian unsigned ints
    words=list(words)
    print(words,' wjjw')
    print(words==[29771398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0])
    print([29771398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0])
    # Initialize state variables
    AA, BB, CC, DD = A, B, C, D

    # Main loop
    for i in range(64):
        if 0 <= i < 16:
            f = F(B, C, D)
            g = i
        elif 16 <= i < 32:
            f = G(B, C, D)
            g = (5 * i + 1) % 16
        elif 32 <= i < 48:
            f = H(B, C, D)
            g = (3 * i + 5) % 16
        elif 48 <= i < 64:
            f = I(B, C, D)
            g = (7 * i) % 16

        # Compute intermediate value
        temp = (A + f + K[i] + words[g]) & 0xFFFFFFFF
        temp = left_rotate(temp, s[i])
        temp = (B + temp) & 0xFFFFFFFF

        # Update state variables
        A, B, C, D = D, temp, B, C

    # Final addition modulo 2^32
    A = (A + AA) & 0xFFFFFFFF
    B = (B + BB) & 0xFFFFFFFF
    C = (C + CC) & 0xFFFFFFFF
    D = (D + DD) & 0xFFFFFFFF

    return A, B, C, D

# MD5 hash function
def md5(input):
    # Preprocess the input
    padded_message = preprocessing_input(input)

    # Initialize state variables
    A, B, C, D = A0, B0, C0, D0

    # Process the message in 512-bit (64-byte) blocks
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        A, B, C, D = md5_compress(A, B, C, D, block)

    # Convert state variables to little-endian bytes and concatenate
    hash_bytes = struct.pack('<4I', A, B, C, D)  # '<4I' means 4 little-endian unsigned ints

    # Return as hex string
    return hash_bytes.hex()
print(md5('abc')) 

b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00'  block .. 
[2153996897, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0]  wjjw
False
[29771398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0]
900150983cd24fb0d6963f7d28e17f72


In [ ]:
def bytes_to_binary_string(byte_data):
    binary_str = ''.join(f'{byte:08b} ' for byte in byte_data)  # Convert each byte to 8-bit binary
    return binary_str.strip()

padded_output = b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00'
binary_representation = bytes_to_binary_string(padded_output)
a22="01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001100000000000000000000000000000000000000000000000000000000000"
print(binary_representation.replace(" ","")==a22)

True


In [ ]:
# Test case for 'abc'
print(preprocessing_input('abc'))


b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00'


In [ ]:
# Initial state variables
A0 = 0x67452301
B0 = 0xefcdab89
C0 = 0x98badcfe
D0 = 0x10325476

# Constants
K = [
    0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
    0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
    0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
    0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821,
    0xf61e2562, 0xc040b340, 0x265e5a51, 0xe9b6c7aa,
    0xd62f105d, 0x02441453, 0xd8a1e681, 0xe7d3fbc8,
    0x21e1cde6, 0xc33707d6, 0xf4d50d87, 0x455a14ed,
    0xa9e3e905, 0xfcefa3f8, 0x676f02d9, 0x8d2a4c8a,
    0xfffa3942, 0x8771f681, 0x6d9d6122, 0xfde5380c,
    0xa4beea44, 0x4bdecfa9, 0xf6bb4b60, 0xbebfbc70,
    0x289b7ec6, 0xeaa127fa, 0xd4ef3085, 0x04881d05,
    0xd9d4d039, 0xe6db99e5, 0x1fa27cf8, 0xc4ac5665,
    0xf4292244, 0x432aff97, 0xab9423a7, 0xfc93a039,
    0x655b59c3, 0x8f0ccc92, 0xffeff47d, 0x85845dd1,
    0x6fa87e4f, 0xfe2ce6e0, 0xa3014314, 0x4e0811a1,
    0xf7537e82, 0xbd3af235, 0x2ad7d2bb, 0xeb86d391
]

# Shift amounts
s = [
    7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21
]

# Logical functions
def F(x, y, z):
    return (x & y) | (~x & z)

def G(x, y, z):
    return (x & z) | (y & ~z)

def H(x, y, z):
    return x ^ y ^ z

def I(x, y, z):
    return y ^ (x | ~z)
def int_to_little_endian_hex(value):
    """ Converts a 32-bit integer to little-endian hex representation. """
    # Extract each byte and reverse the order
    little_endian_bytes = [
        (value >> (8 * i)) & 0xFF  # Extract byte i (0 to 3)
        for i in range(4)  # Loop through 4 bytes
    ]
    
    # Convert each byte to a two-character hex string and join them
    little_endian_hex = ''.join(f'{byte:02x}' for byte in little_endian_bytes)
    
    return little_endian_hex  # Return as a hex string
# Left rotate function
def left_rotate(n, b):
    return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

# Preprocessing function
def number_to_binary_little_endian(n):
    """Convert an integer to a 64-bit binary string in little-endian format."""
    return ''.join(format((n >> (8 * i)) & 0xFF, '08b') for i in range(8))

def preprocessing_input(input):
    """Prepares input for MD5 by adding padding and length encoding (little-endian)."""
    input_bytes = input.encode('utf-8')
    n = len(input_bytes)
    # Convert message to binary
    message_bits = ''.join(format(byte, '08b') for byte in input_bytes)
    # Append '1' bit
    message_bits += '1'
    # Compute k (number of zero bits required)
    k = (448 - ((n * 8 + 1) % 512)) % 512
    message_bits += '0' * k  # Append k zero bits
    # Append length in 64-bit LITTLE-ENDIAN format
    length_bits = number_to_binary_little_endian(n * 8)
    message_bits += length_bits
    return message_bits

# Word generation function
def generate_words(binary_input):
    words = []
    for i in range(16):  # 16 words of 32 bits each
        start = i * 32
        end = start + 32
        word = binary_input[start:end]  # Extract 32-bit chunk
        # Convert binary string to 4 bytes (each byte = 8 bits)
        byte1 = word[0:8]
        byte2 = word[8:16]
        byte3 = word[16:24]
        byte4 = word[24:32]
        # Reverse byte order (little-endian conversion)
        little_endian_binary = byte4 + byte3 + byte2 + byte1  # Swap positions
        # Convert back to integer
        int_word = int(little_endian_binary, 2)
        words.append(int_word)
    return words

# Main MD5 computation
def NthBlock(input,A0,B0,C0,D0):
    # Preprocess input
    M = generate_words(input)
    # Initialize state variables
    A, B, C, D = A0, B0, C0, D0

    # Main loop
    for i in range(64):
        if 0 <= i < 16:
            f = F(B, C, D)
            g = i
        elif 16 <= i < 32:
            f = G(B, C, D)
            g = (5 * i + 1) % 16
        elif 32 <= i < 48:
            f = H(B, C, D)
            g = (3 * i + 5) % 16
        elif 48 <= i < 64:
            f = I(B, C, D)
            g = (7 * i) % 16

        # Compute intermediate value
        temp = (A + f + K[i] + M[g]) & 0xFFFFFFFF
        temp = left_rotate(temp, s[i])
        temp = (B + temp) & 0xFFFFFFFF

        # Update state variables
        A, B, C, D = D, temp, B, C

    # Final addition modulo 2^32
    A0_final = (A0 + A) & 0xFFFFFFFF
    B0_final = (B0 + B) & 0xFFFFFFFF
    C0_final = (C0 + C) & 0xFFFFFFFF
    D0_final = (D0 + D) & 0xFFFFFFFF
    return A0_final , B0_final ,C0_final, D0_final
def MD5_hash(input):
    input_bits=preprocessing_input(input)
    # print(len(input_bits))
    n=len(input_bits)
    for i in range(n//512):
        # print(i)
        if(i==0):
           A,B,C,D= NthBlock(input_bits[i*512:(i+1)*512],A0,B0,C0,D0)
        else:
            A,B ,C ,D =NthBlock(input_bits[i*512:(i+1)*512],A,B,C,D)
    hash_bytes = (
        int_to_little_endian_hex(A) +
        int_to_little_endian_hex(B) +
        int_to_little_endian_hex(C) +
        int_to_little_endian_hex(D)
    )
    return hash_bytes
# print(MD5_hash('abc')) 
# print("f36bb4bcabf143637056f8d2476d4c4b"=="f36bb4bcabf143637056f8d2476d4c4b")
input_txt=input("enter plain text: ")
# input_txt='a'*1024*1024*10
print(MD5_hash(input_txt))
# print("0de4ae65447885e9d10cb8e6a4fc15cd"=="0de4ae65447885e9d10cb8e6a4fc15cd")
print("a33e96f787de75a75c3caa0f05d17152"=="a33e96f787de75a75c3caa0f05d17152")

enter plain text:  jsmx
92b981fe003bec749a5b2299c77d3c24
True


In [9]:
print("e56e104794a18df5f41f6d2d87b4cc67"=="e56e104794a18df5f41f6d2d87b4cc67")

True


In [10]:
with open("test.txt", "w") as file:
    file.write("a" * 1024 * 1024*100)  # 1MB of 'a'


In [ ]:
def int_to_little_endian_hex(value):
    """ Converts a 32-bit integer to little-endian hex representation. """
    # Extract each byte and reverse the order
    little_endian_bytes = [
        (value >> (8 * i)) & 0xFF  # Extract byte i (0 to 3)
        for i in range(4)  # Loop through 4 bytes
    ]
    
    # Convert each byte to a two-character hex string and join them
    little_endian_hex = ''.join(f'{byte:02x}' for byte in little_endian_bytes)
    
    return little_endian_hex  # Return as a hex string

A_final = 0x98500190  
little_endian_hex = int_to_little_endian_hex(A_final)
print(little_endian_hex)  


90015098
